In [96]:
import pandas as pd
import numpy as np
import sklearn.model_selection as Ms
import seaborn as sns
import matplotlib.pyplot as plt

In [97]:
!pip install -U openpyxl

In [98]:
df=pd.read_csv('/content/energydata_complete (1).csv')


In [99]:
import sklearn as sk

In [100]:
df.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [101]:
df1 = df.drop(columns = ['date', 'lights'])

In [102]:
simple_linear_reg_df = df[["T2", 'T6']].sample(15, random_state=2)
simple_linear_reg_df

,T2,T6
1117,17.426667,-4.238889
16275,22.760000,14.690000
13272,21.230000,8.926667
3160,21.100000,7.690000
19210,21.856667,10.800000
8260,17.356667,5.000000
12299,19.000000,5.300000
13505,22.700000,17.133333
12913,19.593333,7.545000
17788,21.890000,9.190000


In [103]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df1), columns=df1.columns)
features_df = normalised_df.drop(columns=['T2'])

T2 = normalised_df['T2']

In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features_df, T2, test_size=0.30, random_state=42)

In [105]:
#initialize the model
linear_model = LinearRegression()

#fit the model to the training dataset
linear_model.fit(x_train, y_train)

LinearRegression()

In [106]:
predicted_values = linear_model.predict(x_test)

#preview predictions
predicted_values

array([0.13427306, 0.42548424, 0.10643315, ..., 0.18781877, 0.35005171,
       0.19315511])

In [107]:
from sklearn.metrics import r2_score

r2_score = r2_score(y_test, predicted_values)

print(round(r2_score, 2))


0.96


In [108]:
#●Residual Sum of Squares (RSS) 
import numpy as np

rss = np.sum(np.square(y_test-predicted_values))
print(round(rss, 2))


5.31


In [109]:
#MAE
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, predicted_values)
print(round(mae, 2))

0.02


In [110]:
#●Root Mean Square Error (RMSE) 
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, predicted_values))

print(round(rmse, 3))

0.03


In [111]:
#Feature selection and lasso regression
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [112]:
linear_model.coef_

array([-2.60846865e-02,  4.92432463e-01,  6.48763608e-01, -1.02015536e+00,
       -2.94272487e-02,  1.01142405e-01,  8.12748788e-02,  1.58540614e-01,
        1.94210982e-02,  1.28391019e-02,  3.05120364e-01,  4.47317692e-03,
       -5.90187328e-02,  4.15505986e-02, -2.63178359e-02, -3.87112557e-02,
        1.28988533e-01,  4.08001801e-03, -2.57215773e-01, -8.64683665e-04,
       -3.77538373e-02, -1.82795988e-02,  3.56032639e-03,  5.89731228e-02,
        4.34776621e-04,  4.34776621e-04])

In [113]:
#Ridge regression
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.4)

In [114]:
reg_predicted_values = ridge_reg.predict(x_test)
reg_predicted_values

array([0.13476264, 0.42182068, 0.1073019 , ..., 0.1885782 , 0.3506487 ,
       0.19414494])

In [115]:
from sklearn.metrics import mean_squared_error

reg_rmse = np.sqrt(mean_squared_error(y_test, reg_predicted_values))

print(round(rmse, 3))

0.03


In [116]:
las_predicted_values = lasso_reg.predict(x_test)
las_predicted_values

array([0.1712569 , 0.29615551, 0.15530418, ..., 0.22322506, 0.36121259,
       0.25298576])

In [117]:
las_rmse = np.sqrt(mean_squared_error(y_test, las_predicted_values))

print(round(rmse, 3))

0.03


In [118]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
    #this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')

In [119]:
non_zero = []
for i in lasso_weights_df['Lasso_weight'].values != 0:
  print(i)

True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True


In [120]:
lasso_weights_df

,Features,Lasso_weight
0,RH_2,-0.469680
1,RH_out,-0.005354
2,Appliances,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Windspeed,-0.000000
6,Press_mm_hg,-0.000000
7,T_out,0.000000
8,RH_9,0.000000
9,T9,0.000000


In [121]:
linear_model_weights.sort_values(by='Linear_Model_Weight')

,Features,Linear_Model_Weight
0,RH_2,-1.020155
1,T_out,-0.257216
2,T7,-0.059019
3,RH_8,-0.038711
4,RH_out,-0.037754
5,T3,-0.029427
6,T8,-0.026318
7,Appliances,-0.026085
8,Windspeed,-0.018280
9,Press_mm_hg,-0.000865
